In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from collections import Counter

# from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, LabelEncoder
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# from xgboost import XGBClassifier

# # 1. Load and explore data
# df = pd.read_csv('Global_Cybersecurity_Threats_2015-2024.csv')
# print("Dataset shape:", df.shape)
# print("\nTarget distribution:")
# print(df['Attack Type'].value_counts())
# print(f"\nTarget distribution percentages:")
# print(df['Attack Type'].value_counts(normalize=True) * 100)

# # Check for missing values
# print("\nMissing values:")
# print(df.isnull().sum())

# # 2. Define features & target
# TARGET = 'Attack Type'
# X = df.drop(columns=[TARGET])
# y = df[TARGET]

# # 3. Encode target labels
# le = LabelEncoder()
# y_enc = le.fit_transform(y)
# print(f"\nClass mapping: {dict(zip(le.classes_, range(len(le.classes_))))}")

# # 4. Split train/test with stratification
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
# )

# print(f"\nTraining set size: {X_train.shape[0]}")
# print(f"Test set size: {X_test.shape[0]}")
# print(f"Training class distribution: {Counter(y_train)}")

# # 5. Feature engineering improvements
# numeric_features = ['Number of Affected Users', 'Incident Resolution Time (in Hours)', 'Year']
# categorical_features = ['Country', 'Target Industry', 'Attack Source', 
#                        'Security Vulnerability Type', 'Defense Mechanism Used']

# # Check if outlier flags exist and include them
# outlier_flags = [col for col in X.columns if 'outlier' in col.lower()]
# if outlier_flags:
#     print(f"\nFound outlier flags: {outlier_flags}")
#     categorical_features.extend(outlier_flags)

# # 6. Improved preprocessors
# numeric_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler()),
# ])

# categorical_transformer = Pipeline([
#     ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')),
# ])

# preprocessor = ColumnTransformer([
#     ('num', numeric_transformer, numeric_features),
#     ('cat', categorical_transformer, categorical_features),
# ], remainder='drop')

# # 7. Improved XGBoost with better parameters for multiclass
# xgb_clf = XGBClassifier(
#     objective='multi:softprob',
#     eval_metric='mlogloss',
#     tree_method='hist',
#     random_state=42,
#     n_jobs=-1,
#     # Better starting parameters
#     n_estimators=200,
#     max_depth=6,
#     learning_rate=0.1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     reg_alpha=0.1,
#     reg_lambda=1.0,
# )

# pipe = Pipeline([
#     ('prep', preprocessor),
#     ('clf', xgb_clf),
# ])

# # 8. Baseline evaluation with multiple metrics
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Test multiple scoring metrics
# scoring_metrics = ['accuracy', 'f1_macro', 'f1_weighted']
# for metric in scoring_metrics:
#     scores = cross_val_score(pipe, X_train, y_train, scoring=metric, cv=cv)
#     print(f'Baseline {metric}: {scores.mean():.3f} ± {scores.std():.3f}')

# # 9. More comprehensive hyperparameter tuning
# param_dist = {
#     'clf__n_estimators': [100, 200, 300, 500],
#     'clf__max_depth': [3, 4, 6, 8],
#     'clf__learning_rate': [0.01, 0.05, 0.1, 0.2],
#     'clf__subsample': [0.6, 0.8, 0.9],
#     'clf__colsample_bytree': [0.6, 0.8, 0.9],
#     'clf__reg_alpha': [0, 0.1, 0.5],
#     'clf__reg_lambda': [0.5, 1.0, 2.0],
#     'clf__min_child_weight': [1, 3, 5],
# }

# print("\nStarting hyperparameter tuning...")
# search = RandomizedSearchCV(
#     pipe, param_distributions=param_dist,
#     n_iter=50,  # Increased iterations
#     cv=cv,
#     scoring='f1_weighted',  # Better metric for imbalanced classes
#     random_state=42, 
#     n_jobs=-1,
#     verbose=1
# )

# search.fit(X_train, y_train)

# print(f'\nBest CV F1-weighted: {search.best_score_:.3f}')
# print('Best params:', search.best_params_)

# # 10. Final evaluation
# best_model = search.best_estimator_
# y_pred = best_model.predict(X_test)
# y_pred_proba = best_model.predict_proba(X_test)

# print(f'\nTest Accuracy: {accuracy_score(y_test, y_pred):.3f}')
# print('\nClassification Report:')
# print(classification_report(y_test, y_pred, target_names=le.classes_))

# # 11. Additional diagnostics
# print("\nConfusion Matrix:")
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

# # Feature importance
# feature_names = (numeric_features + 
#                 list(best_model.named_steps['prep'].named_transformers_['cat']
#                      .named_steps['onehot'].get_feature_names_out(categorical_features)))

# importances = best_model.named_steps['clf'].feature_importances_
# feature_importance_df = pd.DataFrame({
#     'feature': feature_names,
#     'importance': importances
# }).sort_values('importance', ascending=False)

# print("\nTop 10 Most Important Features:")
# print(feature_importance_df.head(10))

# # 12. Prediction confidence analysis
# print("\nPrediction Confidence Analysis:")
# max_probas = np.max(y_pred_proba, axis=1)
# print(f"Average prediction confidence: {max_probas.mean():.3f}")
# print(f"Predictions with confidence > 0.5: {(max_probas > 0.5).sum()}/{len(max_probas)}")
# print(f"Predictions with confidence > 0.8: {(max_probas > 0.8).sum()}/{len(max_probas)}")

# # 13. Recommendations based on results
# print("\n" + "="*60)
# print("DIAGNOSTIC RECOMMENDATIONS:")
# print("="*60)

# if accuracy_score(y_test, y_pred) < 0.3:
#     print("⚠️  Very low accuracy detected. Possible issues:")
#     print("   - Data quality problems (mislabeled samples)")
#     print("   - Insufficient features for discrimination")
#     print("   - High class imbalance or noise")
#     print("   - Consider data cleaning and feature engineering")

# if max_probas.mean() < 0.4:
#     print("⚠️  Low prediction confidence suggests:")
#     print("   - Model uncertainty due to overlapping classes")
#     print("   - Consider ensemble methods or different algorithms")
#     print("   - Review feature selection and engineering")

# # Calculate class balance
# class_counts = Counter(y_train)
# min_class = min(class_counts.values())
# max_class = max(class_counts.values())
# imbalance_ratio = max_class / min_class

# if imbalance_ratio > 2:
#     print(f"⚠️  Class imbalance detected (ratio: {imbalance_ratio:.1f})")
#     print("   - Consider using class_weight='balanced' in XGBoost")
#     print("   - Try SMOTE or other resampling techniques")
#     print("   - Use stratified sampling more carefully")

# print("\n💡 IMPROVEMENT SUGGESTIONS:")
# print("   1. Examine data quality and labeling accuracy")
# print("   2. Try feature selection/engineering")
# print("   3. Consider ensemble methods")
# print("   4. Experiment with different algorithms (Random Forest, Neural Networks)")
# print("   5. Use cross-validation with different metrics")

# # 14. Additional data exploration
# print("\n" + "="*60)
# print("ADDITIONAL DATA INSIGHTS:")
# print("="*60)

# # Check feature correlation with target
# print("\nNumeric features summary:")
# for col in numeric_features:
#     if col in df.columns:
#         print(f"{col}: mean={df[col].mean():.2f}, std={df[col].std():.2f}, "
#               f"min={df[col].min():.2f}, max={df[col].max():.2f}")

# # Check categorical feature cardinality
# print("\nCategorical features cardinality:")
# for col in categorical_features:
#     if col in df.columns and 'outlier' not in col.lower():
#         unique_vals = df[col].nunique()
#         print(f"{col}: {unique_vals} unique values")
#         if unique_vals < 20:  # Show values if not too many
#             print(f"   Values: {list(df[col].unique())}")

# # Save detailed results for further analysis
# results_dict = {
#     'test_accuracy': accuracy_score(y_test, y_pred),
#     'best_cv_score': search.best_score_,
#     'best_params': search.best_params_,
#     'feature_importance': feature_importance_df.to_dict(),
#     'class_distribution': dict(Counter(y_train)),
#     'prediction_confidence': {
#         'mean': max_probas.mean(),
#         'confident_predictions_50': (max_probas > 0.5).sum(),
#         'confident_predictions_80': (max_probas > 0.8).sum(),
#         'total_predictions': len(max_probas)
#     }
# }

# print(f"\nModel saved results summary:")
# print(f"Final test accuracy: {results_dict['test_accuracy']:.3f}")
# print(f"Best CV F1-weighted: {results_dict['best_cv_score']:.3f}")

# # 15. Try a simple baseline for comparison
# from sklearn.dummy import DummyClassifier
# dummy_clf = DummyClassifier(strategy='most_frequent', random_state=42)
# dummy_clf.fit(X_train, y_train)
# dummy_pred = dummy_clf.predict(X_test)
# dummy_accuracy = accuracy_score(y_test, dummy_pred)
# print(f"\nBaseline (most frequent class) accuracy: {dummy_accuracy:.3f}")
# print(f"Model improvement over baseline: {(results_dict['test_accuracy'] - dummy_accuracy):.3f}")

# if results_dict['test_accuracy'] <= dummy_accuracy + 0.05:
#     print("⚠️  WARNING: Model barely outperforms random baseline!")
#     print("   This suggests the features may not be predictive of the target.")

In [ ]:
# import pandas as pd

# df = pd.read_csv("Global_Cybersecurity_Threats_2015-2024.csv")

# # Drop rows where Attack Type (the label) is missing
# df = df.dropna(subset=["Attack Type"]).reset_index(drop=True)

# # Optional: remove obvious duplicates
# df = df.drop_duplicates()

# from sklearn.model_selection import train_test_split

# X = df.drop(columns=["Attack Type"])
# y = df["Attack Type"]

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.20, stratify=y, random_state=42
# )

# from catboost import CatBoostClassifier

# # Identify categorical feature indices (CatBoost needs positions, not names)
# cat_cols = X.select_dtypes(include="object").columns
# cat_features = [X.columns.get_loc(c) for c in cat_cols]

# clf = CatBoostClassifier(
#     iterations      = 1500,        # plenty of trees; use early stopping
#     learning_rate   = 0.03,
#     depth           = 6,
#     loss_function   = "MultiClass",
#     eval_metric     = "TotalF1",    # macro F1 = good for 6-class target
#     random_seed     = 42,
#     auto_class_weights = "Balanced",
#     cat_features    = cat_features,
#     verbose         = 100
# )

# clf.fit(
#     X_train, y_train,
#     eval_set=(X_test, y_test),
#     early_stopping_rounds=100
# )

# from sklearn.metrics import classification_report, confusion_matrix

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred, digits=3))

# import matplotlib.pyplot as plt

# feat_imp = clf.get_feature_importance(prettified=True)
# feat_imp.head(15).plot(kind="barh", x="Feature Id", y="Importances", figsize=(6,6))
# plt.title("CatBoost – Feature Importance (Top 15)")
# plt.tight_layout()
# plt.show()


# # # 2) Model
# # clf = CatBoostClassifier(
# #     iterations=1500, learning_rate=0.03, depth=6, loss_function="MultiClass",
# #     eval_metric="TotalF1", random_seed=42, auto_class_weights="Balanced",
# #     cat_features=cat_features, verbose=100
# # )

# # # 3) Fit + early stopping
# # clf.fit(X_train, y_train,
# #         eval_set=(X_test, y_test),
# #         early_stopping_rounds=100)

# # # 4) Metrics
# # from sklearn.metrics import classification_report
# # print(classification_report(y_test, clf.predict(X_test), digits=3))
